This notebooks improves from the previous version by changing most of the fors into numpy matrix multiplication on:<br>
-feedForward 
-backProp

Advantages:<br>
-It's a lot faster<br>

The focus of the previous notebooks were understanding the operations involved in neural networks, focusing more on the code and mechanisms involved than on the math. On this notebook, we utilize matrix operations to make it faster and more concise, and if any of the operations seem strange, just return to the previous notebook and compare the code, the math is the same at the end. <br>
This notebook was made mostly following Michael Nielsen's book http://neuralnetworksanddeeplearning.com/

In [51]:
import numpy as np
import random as rd
from keras.datasets import mnist



In [52]:
class Network:
    """
    The main object we're going to use accross this notebook
    It's a neural network that takes as input a list of 
    layers nodes
    
    Ex: [2, 3, 1] is a 3 layer network, with 2 neurons of input, 3 neurons 
    in the hidden layer and 1 for the output layer
    
    Supposedly it can take more than just 3 layers but I didnt test it
    
    It initializes an object with the proper weights, biases, activations and z
    based on the layers list. It also has the layers list and the number of layers
    
    The weights and biases initialized following a Gaussian with standard deviation 1/sqrt(n_in)
    with n_in = number of weights into the neuron
    """
    def __init__(self, layers: list):        
        np.random.seed(42)        
        b = []
        w = []
        a = []
        z = []
        for l in range(0, len(layers)):
            # skipping one layer for the weights and biases
            if (l+1) < len(layers):
                b.append(np.random.normal(loc=0, scale=1,size=layers[l+1]))
                wScale = 1/np.sqrt(layers[l])
                w.append(np.random.normal(loc=0,scale=wScale,size=[layers[l],layers[l+1]]))
                #print(w[l])
            a.append(np.zeros(layers[l]))
            z.append(np.zeros(layers[l]))
        # b[i][j] -> "i" is which layer, "j" which neuron
        # w[i][j][k] -> "i" is which layer, "j" which neuron of the first layer, "k" which neuron of the second layer
        self.b = b
        self.w = w
        self.a = a
        self.z = z
        self.nLayers = len(layers)
        self.layers = layers
        
    @staticmethod
    def copy(net):
        copiedNet = Network([784,30,10])
        copiedNet.a = np.copy(net.a)
        copiedNet.z = np.copy(net.z)
        for l in range(2):
            copiedNet.w[l] = np.copy(net.w[l])
            copiedNet.b[l] = np.copy(net.b[l])
        return copiedNet
            

In [53]:
def sigmoid(n: float):
    return 1.0/(1.0+np.exp(-n))

def sigmoid_derivative(n: float):
    """Derivative of the sigmoid function."""
    return sigmoid(n)*(1-sigmoid(n))
    
    

In [54]:
def feedForward(net: Network) -> Network:
    """
    Feedforwading the activations to the next layer
    
    It will take as input the network already with the input image as the activation 
    on the first layer and then feedforward to the next layrse
    
    It returns the network with all the activations set
    """
    
    for l in range(0, net.nLayers-1):
        net.z[l+1] = np.dot(np.transpose(net.a[l]), net.w[l]) + net.b[l]
        net.a[l+1] = sigmoid(net.z[l+1])
            
    return net
    
    

In [55]:
def setInput(net: Network, MNISTnumber):
    """
    Inputs the MNIST number into the network, since the number is a 28x28 matrix, 
    we transform it into a 784 array
    
    We also scale the pixels as to be between 0 and 1 for the sigmoid function 
    instead of 0 and 255
    
    Returns the network with the proper activations on all layers since it pass 
    through the feedforward step
    """
    numberArr = np.asarray(MNISTnumber).flatten()
    # scaling the array so that the range is between 0 and 1
    numberArr = np.interp(numberArr, (numberArr.min(), numberArr.max()), (0, 1))
    net.z[0] = numberArr
    net.a[0] = numberArr
    net = feedForward(net)
    
    return net

In [56]:
def testNetwork(net: Network, test_X, test_y, nTests: int):
    """
    A function to test our network
    
    It returns the overall accuracy and the numbers our network guessed
    """
    
    correctOutput = 0
    X = test_X[:nTests]
    y = test_y[:nTests]
    outputs = np.zeros(10)
    for i in range(nTests):
        net = setInput(net, X[i])
        networkOutput = np.argmax(net.a[-1])
        outputs[networkOutput] += 1
        #print(f"number: {y[i]}, networkOutput: {networkOutput}, activations: {net.a[-1]}")
        if y[i] == networkOutput:
            correctOutput += 1
    acc = correctOutput/nTests
    return acc, outputs


In [57]:
def gridSearch(train_X, train_y, test_X, test_y, batchSize: int, learningRates: list, epochs: int, lamb):
    """
    A function to perform a gridSearch in order to find the best learningRates

    It takes as input the network, the training images of MNIST, the training labels,
    the test images, the test labels, the batchSize for SGD,
    a list of learningRates as to find the best inside the list
    the number of epochs to perform SGD
    
    
    It returns the best network accross all learning rates list
    """
    bestAcc = 0
    for eta in learningRates:
        # resetting the network
        net = Network([784,30,10])
        net = SGD(net, train_X, train_y, batchSize=batchSize, nEpochs=epochs, learningRate=eta, lamb=lamb)
        acc, outputs = testNetwork(net, test_X, test_y, batchSize) 
        if acc > bestAcc:
            bestNet = net
            bestAcc = acc
    return bestNet


The list below is all equations that were used to compute the erros and then propagate through the network:

To calculate the error on the last layer: 
$$\delta^L = (a^L - y)\odot \sigma'(z^L)$$

To calculate the error on the other layers:
$$\delta^l = ((w^{l+1})^T\delta^{l+1})\odot \sigma'(z^l)$$

To repass the error to the bias: 
$$\frac{\partial C}{\partial b^l_j} = \delta^l_j$$

To repass the error to the weights:
$$\frac{\partial C}{\partial w^l_{jk}} = a^{l-1}_k\delta^l_j$$

In [58]:
def backProp(net: Network, y) -> Network:
    """
    The backpropagation step: first we calculate the error on the last layer, 
    then we pass to the previous layers all the while applying the error 
    to the weights and biases. Here we used Cross-entropy as our cost function
    
    Example on a 3 layer network: We calculate the error on the last layer, 
    apply it to the last layer's weights and biases, and then calculate the 
    error on the next layer, propagate to the weights and biases and it's done
    
    It takes as input the network and the label of the number the network was activated on
    
    It returns the modifications to the weights and biases (nablaW and nablaB) 
    the network should have
    """
    layers = net.layers
    nablaB = [np.zeros(i.shape) for i in net.b]
    nablaW = [np.zeros(i.shape) for i in net.w]
    delta = np.array(net.a[-1]) - 0
    delta[y] = net.a[-1][y] - 1
    for l in range(net.nLayers-1, 0, -1):
        #nablaB and nablaW have -1 because they only have 2 layers instead of 3
        nablaB[l-1] = delta
                
        nablaW[l-1] = np.outer(net.a[l-1], delta)        
        # finding the error one layer behind
        # in the book it needs a transpose because its weight[layer][receivingNeuron][givingNeuron]
        # but my implementation uses weight[layer][givingNeuron][receivingNeuron] so it's not necessary
        delta = (np.dot(net.w[l-1], delta))*sigmoid_derivative(net.z[l-1])
        
    return nablaB, nablaW


In [62]:
def SGD(net: Network, X: list, y: list, batchSize: int, nEpochs: int, learningRate, lamb, earlyStop=False) -> Network:
    """
    Implementation of Stochastic Gradient Descent
    
    It takes as input the network, the MNIST dataset, the MNIST labels of the dataset, 
    the size of the batch to do gradient descent, the number of epochs it should run,
    the learning rate eta (I found the best eta to be in the order of 1s)
    and the regularization term lambda
    
    It returns a trained network
    """
    bestAcc = 0
    bestEpoch = 0
    earlyNet = net
    eta = learningRate
    etaChangeEpoch = 0
    for epoch in range(nEpochs):
        batch = rd.sample(range(len(X)), batchSize)
        nablaB = [np.zeros(i.shape) for i in net.b]
        nablaW = [np.zeros(i.shape) for i in net.w]
        for i in batch:
            net = setInput(net, X[i])
            # finding what should be modified based on this particular example
            deltaNablaB, deltaNablaW = backProp(net, y[i])
            # passing this modifications to our overall modifications matrices
            for l in range(net.nLayers-1):
                nablaB[l] += deltaNablaB[l]
                nablaW[l] += deltaNablaW[l]
        
        # applying the changes to our network
        for l in range(net.nLayers-1):
            net.b[l] = net.b[l] - eta * (nablaB[l]/batchSize) 
            net.w[l] = net.w[l] - eta * (nablaW[l]/batchSize) - eta * (lamb/batchSize) *  net.w[l]
        acc, outputs = testNetwork(net, X, y, nTests=batchSize)
        if acc > bestAcc:
            bestAcc = acc
            bestEpoch = epoch
            earlyNet = Network.copy(net)
            etaChangeEpoch = epoch
        if epoch%10 == 0:
            print(f'learningRate: {learningRate} epochs: {epoch} acc: {acc}, outputs: {outputs}')
    print(f'best acc: {bestAcc} on epoch: {bestEpoch}')
    if earlyStop:
        return earlyNet
    return net
        

In [60]:
# loading the dataset
(train_X, train_y), (test_X, test_y) = mnist.load_data()

In [78]:
net = gridSearch(train_X, train_y, test_X, test_y, batchSize=100, learningRates=[2], epochs=200, lamb=0.1)

learningRate: 2 epochs: 0 acc: 0.06, outputs: [  0.   0. 100.   0.   0.   0.   0.   0.   0.   0.]
learningRate: 2 epochs: 100 acc: 0.92, outputs: [14. 13.  6. 12. 11.  4. 12. 11.  8.  9.]
learningRate: 2 epochs: 200 acc: 0.94, outputs: [14. 13.  5. 13. 11.  5. 11. 12.  8.  8.]
best acc: 0.97 on epoch: 279


In [79]:
testNetwork(net, test_X, test_y, len(test_X))

(0.9189,
 array([1001., 1166.,  956., 1046.,  991.,  893.,  997.,  971.,  969.,
        1010.]))

In [ ]:
# Seeing our network in action

import matplotlib.pyplot as plt

# pick a sample to plot
sample = 50099
image = train_X[sample]
# plot the sample
fig = plt.figure
plt.imshow(image, cmap='gray')
plt.show()

net = setInput(net, train_X[sample])
networkOutput = np.argmax(net.a[-1])
networkOutput